# Notebook 4: Advanced Prompting Techniques

This notebook covers advanced prompting techniques:
- Zero-shot vs few-shot prompting
- Chain-of-thought reasoning
- When reasoning helps vs hurts
- Prompt chaining and decomposition

## Setup

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Optional
import json

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

## Part 1: Zero-Shot vs Few-Shot Prompting

### Zero-Shot: No Examples

In [2]:
# Zero-shot: Just instructions
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "Classify the sentiment of product reviews as positive, negative, or neutral."
        },
        {
            "role": "user",
            "content": "This tent is okay, nothing special but it works."
        }
    ],
    temperature=0
)

print("🎯 Zero-shot classification:")
print(response.choices[0].message.content)

🎯 Zero-shot classification:
Neutral


### Few-Shot: Learning by Example

Provide examples to establish a pattern.

In [3]:
# Few-shot: Include examples
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "Classify the sentiment of product reviews as positive, negative, or neutral."
        },
        # Example 1
        {"role": "user", "content": "This backpack is amazing! Super comfortable and durable."},
        {"role": "assistant", "content": "positive"},
        # Example 2
        {"role": "user", "content": "Terrible quality. Broke after one use."},
        {"role": "assistant", "content": "negative"},
        # Example 3
        {"role": "user", "content": "It's fine. Does what it's supposed to do."},
        {"role": "assistant", "content": "neutral"},
        # Actual query
        {"role": "user", "content": "This tent is okay, nothing special but it works."}
    ],
    temperature=0
)

print("📚 Few-shot classification:")
print(response.choices[0].message.content)

📚 Few-shot classification:
neutral


### Why Few-Shot Works

Examples establish:
- Output format
- Classification criteria
- Edge case handling
- Consistent style

### Few-Shot with Structured Output

In [4]:
class SentimentAnalysis(BaseModel):
    sentiment: str = Field(..., description="positive, negative, or neutral")
    confidence: float = Field(..., ge=0, le=1, description="Confidence score 0-1")
    reasoning: str = Field(..., description="Brief explanation")

# Few-shot with structured output
examples = [
    {
        "input": "This backpack is amazing! Super comfortable.",
        "output": {"sentiment": "positive", "confidence": 0.95, "reasoning": "Strong positive language (amazing, super)"}
    },
    {
        "input": "Terrible quality. Broke immediately.",
        "output": {"sentiment": "negative", "confidence": 0.98, "reasoning": "Explicit negative words and product failure"}
    },
    {
        "input": "It's okay, does the job.",
        "output": {"sentiment": "neutral", "confidence": 0.85, "reasoning": "Lukewarm language, neither praise nor complaint"}
    }
]

# Build few-shot prompt
examples_text = "\n\n".join([
    f"Input: {ex['input']}\nOutput: {json.dumps(ex['output'])}"
    for ex in examples
])

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": f"""Analyze sentiment of product reviews.
Return JSON matching this schema: {json.dumps(SentimentAnalysis.model_json_schema())}

Examples:
{examples_text}"""
        },
        {
            "role": "user",
            "content": "This tent is okay, nothing special but it works."
        }
    ],
    response_format={"type": "json_object"},
    temperature=0
)

result = SentimentAnalysis(**json.loads(response.choices[0].message.content))
print("📊 Few-shot structured analysis:")
print(result.model_dump_json(indent=2))

📊 Few-shot structured analysis:
{
  "sentiment": "neutral",
  "confidence": 0.87,
  "reasoning": "The review expresses a lack of strong opinion, indicating it functions adequately but does not exceed expectations."
}


### When to Use Few-Shot

| Use Case | Zero-Shot | Few-Shot |
|----------|-----------|----------|
| Well-known tasks (summarization, translation) | ✅ | Optional |
| Custom formats or classifications | ❌ | ✅ |
| Domain-specific terminology | ❌ | ✅ |
| Edge cases need handling | ❌ | ✅ |

**Tradeoff:** Few-shot increases token usage (cost + latency).

## Part 2: Chain-of-Thought Reasoning

Ask the model to "think step by step" for complex problems.

### Without Chain-of-Thought

In [5]:
# Direct answer (no reasoning)—ask for final answer only so model doesn't default to steps
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "Answer with only the final number or one short sentence. Do not show steps or reasoning."
        },
        {
            "role": "user",
            "content": """A camping store has a 20% off sale. A tent originally costs $350.
If I have a $50 gift card, how much will I pay?"""
        }
    ],
    temperature=0
)

print("❌ Without chain-of-thought:")
print(response.choices[0].message.content)
print()

❌ Without chain-of-thought:
$250.



### With Chain-of-Thought

In [6]:
# Chain-of-thought reasoning
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Think step by step and show your reasoning."
        },
        {
            "role": "user",
            "content": """A camping store has a 20% off sale. A tent originally costs $350.
If I have a $50 gift card, how much will I pay?

Let's solve this step by step:"""
        }
    ],
    temperature=0
)

print("✅ With chain-of-thought:")
print(response.choices[0].message.content)

✅ With chain-of-thought:
Sure! Let's break this down step by step.

1. **Calculate the discount amount**: 
   The tent is originally priced at $350, and there is a 20% discount. To find the discount amount, we can use the formula:
   \[
   \text{Discount} = \text{Original Price} \times \text{Discount Rate}
   \]
   Here, the discount rate is 20%, which can be expressed as a decimal (0.20). So:
   \[
   \text{Discount} = 350 \times 0.20 = 70
   \]

2. **Calculate the sale price of the tent**: 
   Now, we subtract the discount from the original price to find the sale price:
   \[
   \text{Sale Price} = \text{Original Price} - \text{Discount}
   \]
   So:
   \[
   \text{Sale Price} = 350 - 70 = 280
   \]

3. **Apply the gift card**: 
   You have a $50 gift card that can be used towards the purchase. We subtract the value of the gift card from the sale price:
   \[
   \text{Amount to Pay} = \text{Sale Price} - \text{Gift Card Value}
   \]
   So:
   \[
   \text{Amount to Pay} = 280 - 50 = 2

### Structured Chain-of-Thought

In [7]:
class ReasoningResult(BaseModel):
    steps: List[str] = Field(..., description="Reasoning steps")
    final_answer: str = Field(..., description="Final answer")

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": f"""Solve problems step by step.
Return JSON with this schema: {json.dumps(ReasoningResult.model_json_schema())}

Include each reasoning step and the final answer."""
        },
        {
            "role": "user",
            "content": """A camping store has a 20% off sale. A tent originally costs $350.
If I have a $50 gift card, how much will I pay?"""
        }
    ],
    response_format={"type": "json_object"},
    temperature=0
)

result = ReasoningResult(**json.loads(response.choices[0].message.content))
print("🧠 Structured reasoning:")
print(result.model_dump_json(indent=2))

🧠 Structured reasoning:
{
  "steps": [
    "Calculate the discount amount on the tent: 20% of $350 is calculated as follows: 0.20 * 350 = $70.",
    "Subtract the discount from the original price: $350 - $70 = $280.",
    "Subtract the gift card amount from the discounted price: $280 - $50 = $230."
  ],
  "final_answer": "$230"
}


### When Reasoning Hurts Performance

Chain-of-thought adds latency and cost. Sometimes a direct answer is better.

In [8]:
# Simple classification doesn't need reasoning
import time

# With reasoning (slower)
start = time.time()
response_with = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "Classify sentiment. Think step by step."
        },
        {"role": "user", "content": "This product is terrible!"}
    ],
    temperature=0
)
time_with = time.time() - start

# Without reasoning (faster)
start = time.time()
response_without = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "Classify sentiment as positive, negative, or neutral. One word only."
        },
        {"role": "user", "content": "This product is terrible!"}
    ],
    temperature=0
)
time_without = time.time() - start

print("⏱️ Performance comparison:")
print(f"With reasoning: {time_with:.2f}s - {len(response_with.choices[0].message.content)} chars")
print(f"Without: {time_without:.2f}s - {len(response_without.choices[0].message.content)} chars")
print(f"\nSpeedup: {time_with/time_without:.1f}x faster without reasoning")

⏱️ Performance comparison:
With reasoning: 2.16s - 673 chars
Without: 1.42s - 8 chars

Speedup: 1.5x faster without reasoning


**Key Insight:** Use chain-of-thought for complex reasoning, not simple tasks.

## Part 3: Prompt Chaining and Decomposition

Break complex tasks into multiple stages.

<img src="img/n4_img_1.png" alt="Prompt Chaining and Decomposition" width="560" style="max-width: 100%; height: auto;">

### Single Prompt (Overloaded)

In [9]:
# ❌ Trying to do too much in one prompt
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": """Read this product review, extract the product name, 
classify the sentiment, identify key issues mentioned, 
and generate a response from customer support.

Review: "The Alpine Pro Tent is disappointing. The zippers broke after one trip 
and it's not as waterproof as advertised. Customer service was unhelpful."""
        }
    ],
    temperature=0
)

print("❌ Overloaded single prompt:")
print(response.choices[0].message.content)
print("\nProblem: Output is unstructured and hard to parse.")

❌ Overloaded single prompt:
**Product Name:** Alpine Pro Tent

**Sentiment Classification:** Negative

**Key Issues Mentioned:**
1. Zippers broke after one trip.
2. Not as waterproof as advertised.
3. Unhelpful customer service.

**Response from Customer Support:**

Dear Valued Customer,

Thank you for taking the time to share your feedback regarding the Alpine Pro Tent. We are truly sorry to hear about your experience with the zippers and the waterproofing, as well as the difficulties you faced with our customer service. Your satisfaction is very important to us, and we take your concerns seriously.

We would like to investigate this matter further and see how we can assist you. Please reach out to us directly at [customer support email/phone number], and we will do our best to resolve these issues for you. 

Thank you for bringing this to our attention, and we hope to make this right.

Best regards,  
[Your Name]  
Customer Support Team  
Alpine Pro

Problem: Output is unstructured a

### Chained Prompts (Better)

In [10]:
review_text = """The Alpine Pro Tent is disappointing. The zippers broke after one trip 
and it's not as waterproof as advertised. Customer service was unhelpful."""

# Stage 1: Extract structured information
class ReviewAnalysis(BaseModel):
    product_name: str
    sentiment: str
    issues: List[str]

response1 = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": f"""Extract information from product reviews.
Return JSON: {json.dumps(ReviewAnalysis.model_json_schema())}"""
        },
        {"role": "user", "content": review_text}
    ],
    response_format={"type": "json_object"},
    temperature=0
)

analysis = ReviewAnalysis(**json.loads(response1.choices[0].message.content))
print("✅ Stage 1 - Analysis:")
print(analysis.model_dump_json(indent=2))
print()

✅ Stage 1 - Analysis:
{
  "product_name": "Alpine Pro Tent",
  "sentiment": "negative",
  "issues": [
    "zippers broke after one trip",
    "not as waterproof as advertised",
    "unhelpful customer service"
  ]
}



In [11]:
# Stage 2: Generate customer support response
response2 = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": """You are a customer support agent.
Write empathetic, helpful responses to customer complaints.
Acknowledge issues and offer solutions."""
        },
        {
            "role": "user",
            "content": f"""Generate a response to this customer review:

Product: {analysis.product_name}
Sentiment: {analysis.sentiment}
Issues: {', '.join(analysis.issues)}

Original review: {review_text}"""
        }
    ],
    temperature=0.7
)

print("✅ Stage 2 - Support Response:")
print(response2.choices[0].message.content)

✅ Stage 2 - Support Response:
Dear [Customer's Name],

Thank you for taking the time to share your experience with the Alpine Pro Tent. I’m truly sorry to hear that it did not meet your expectations and that you encountered these issues with the zippers and waterproofing. Your feedback is invaluable, and I want to assure you that we take it seriously.

It’s frustrating to invest in a product only to face these problems, especially during your first trip. I understand how important it is to have reliable gear when you’re out enjoying the great outdoors. Regarding the customer service experience you had, I sincerely apologize for not meeting your needs. We strive to provide support that is both effective and empathetic, and I regret that we fell short in this instance.

To help resolve this, I would like to offer you a replacement tent or a full refund, depending on what you prefer. Additionally, I will ensure that your feedback is escalated to our product team, as we are always looking 

**Benefits of Chaining:**
- Each stage has a clear, focused task
- Structured intermediate data
- Can cache or reuse intermediate results

### Multi-Stage Pipeline

In [12]:
def process_review_pipeline(review: str) -> dict:
    """Multi-stage review processing pipeline."""
    
    # Stage 1: Extract and analyze
    response1 = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": f"""Extract information from reviews.
Return JSON: {json.dumps(ReviewAnalysis.model_json_schema())}"""
            },
            {"role": "user", "content": review}
        ],
        response_format={"type": "json_object"},
        temperature=0
    )
    analysis = ReviewAnalysis(**json.loads(response1.choices[0].message.content))
    
    # Stage 2: Determine priority
    priority = "high" if analysis.sentiment == "negative" and len(analysis.issues) > 2 else "normal"
    
    # Stage 3: Generate response
    response2 = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": f"""You are a customer support agent.
Priority level: {priority}
Write empathetic responses that acknowledge issues and offer solutions."""
            },
            {
                "role": "user",
                "content": f"""Respond to this review:
Product: {analysis.product_name}
Issues: {', '.join(analysis.issues)}
Original: {review}"""
            }
        ],
        temperature=0.7
    )
    
    return {
        "analysis": analysis.model_dump(),
        "priority": priority,
        "response": response2.choices[0].message.content
    }

# Test pipeline
result = process_review_pipeline(review_text)
print("🔄 Complete Pipeline Result:")
print(json.dumps(result, indent=2))

🔄 Complete Pipeline Result:
{
  "analysis": {
    "product_name": "Alpine Pro Tent",
    "sentiment": "negative",
    "issues": [
      "zippers broke after one trip",
      "not as waterproof as advertised",
      "unhelpful customer service"
    ]
  },
  "priority": "high",
  "response": "Dear [Customer's Name],\n\nI sincerely apologize for the experience you've had with the Alpine Pro Tent. It\u2019s disheartening to hear that the zippers broke after just one trip, and I understand how frustrating it must be to deal with waterproofing issues, especially when you were counting on it for your outdoor adventures. \n\nAdditionally, I\u2019m sorry to learn that our customer service did not meet your expectations. Your feedback is incredibly valuable, and we want to ensure that we address your concerns effectively.\n\nTo help resolve these issues, I would like to offer you a replacement tent or a full refund, depending on what you prefer. We are also committed to improving our customer su